# **Random Walk Model: 확산 현상의 시뮬레이션**

(이 노트북은 **제주대학교 화학과 물리화학 대학원 수업**의 일환으로 작성되었습니다.)

**Random Walk**는 모델은 입자의 이동을 무작위적인 방식으로 묘사하는 간단하면서도 강력한 수학적 도구입니다.

이 모델은 물리학, 화학, 금융, 생물학 등 다양한 분야에서 확산, 주가의 변동, 동물의 이동 경로, 단백질 탐색 행동 등 무작위성이 관여하는 현상들을 설명하는 데 널리 활용됩니다.

**목적**

이 노트북은 물리화학 대학원 수업의 일환으로, 화학에서 자주 등장하는 확산(diffusion) 개념을 보다 직관적이고 체계적으로 이해하기 위해 작성되었습니다.
간단한 시뮬레이션을 통해 다음과 같은 내용을 다룹니다:

1차원 및 2차원 Random Walk 모델 구현

입자 수와 시간에 따른 평균 위치와 분산의 변화 분석

확률 밀도 함수 (PDF) 를 이용한 위치 분포의 시각화

스텝 크기(step_size)가 확산 특성에 미치는 영향

**학습 포인트**

* 무작위 운동(randomness)이 누적되며 어떻게 예측 가능한 통계적 패턴(예: 정규분포)을 만들어내는지 확인

* 확산 계수(Diffusion Coefficient)와 분산 간의 이론적 관계
𝜎$^2$=2𝐷𝑡 를 시뮬레이션 결과와 비교

* 시뮬레이션을 통해 이론 모델과 실제 수치 계산의 연결고리를 체험

**활용 팁**

각 셀을 직접 실행해보며 파라미터(step_size, n_particles, n_steps)를 바꿔보면 다양한 확산 조건을 손쉽게 실험해볼 수 있습니다.
또한 matplotlib을 활용한 시각화를 통해 복잡한 경로를 직관적으로 이해할 수 있습니다.

1. 아래 시뮬레이션은 **2차원 무작위 보행(2D Random Walk)**을 시뮬레이션하고, 시간의 흐름에 따라 색상이 변하는 궤적을 시각화하는 예제입니다.



*   **랜덤 워크(Random Walk)**란 입자가 일정한 규칙 없이 무작위로 이동하는 과정을 의미합니다.
*   본 시뮬레이션에서는 입자가 매 스텝마다 상하좌우 방향 중 하나로 1만큼 이동하며, 총 nstep 만큼 이동합니다.
*   초기 위치는 원점 (0, 0)이며, 시간의 흐름에 따라 입자의 궤적을 기록합니다.





In [ ]:
import random
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import numpy as np

nstep = 1000

X = [0]
Y = [0]
xt = 0
yt = 0

# 랜덤 워크 생성
for _ in range(nstep):
    xstep = random.choice([-1, 1])
    ystep = random.choice([-1, 1])
    xt += xstep
    yt += ystep
    X.append(xt)
    Y.append(yt)

# 선분들 만들기 (각 두 점씩 쌍을 이루도록)
points = np.array([X, Y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

# 색상 지정: 시간에 따라 선 색 바꾸기
colors = np.linspace(0, 1, len(segments))

# LineCollection 생성
lc = LineCollection(segments, cmap='viridis', norm=plt.Normalize(0, 1))
lc.set_array(colors)
lc.set_linewidth(2)

# 플롯
fig, ax = plt.subplots(figsize=(8, 6))
ax.add_collection(lc)

# 시작점 마커
ax.plot(0, 0, marker='o', color='red', markersize=8, label='Start (0,0)')

# 마지막 점 마커
ax.plot(X[-1], Y[-1], marker='X', color='black', markersize=8, label='End')

# 컬러바
cbar = plt.colorbar(lc, ax=ax)
cbar.set_label('Step Index (Normalized)')

# 그래프 설정
ax.set_xlim(min(X) - 10, max(X) + 10)
ax.set_ylim(min(Y) - 10, max(Y) + 10)
ax.set_aspect('equal')
ax.set_title("2D Random Walk with Color Gradient")
ax.set_xlabel("X")
ax.set_ylabel("Y")
plt.show()


아래 시뮬레이션은 **여러 입자들이 독립적으로 2차원 무작위 보행(Random Walk)**을 수행하는 모습을 시각화한 것입니다.
각 입자는 동일한 규칙에 따라 무작위로 움직이며, 각기 다른 색상으로 표시되어 궤적이 쉽게 구분됩니다.

*   n_particles: 총 입자의 개수.
*   n_steps: 각 입자의 이동 횟수.
*   각 스텝마다 입자는 +1 또는 -1 방향으로 X축과 Y축 각각에 대해 독립적으로 움직입니다.
*   초기 위치는 모든 입자가 (0, 0)입니다.

In [ ]:
import random
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import numpy as np

n_particles = 10  # 입자 수
n_steps = 500     # 스텝 수

fig, ax = plt.subplots(figsize=(10, 8))

# 컬러맵에서 입자 수만큼 색 고르기
cmap = plt.cm.get_cmap('tab10', n_particles)
colors = [cmap(i) for i in range(n_particles)]

# 모든 입자의 마지막 위치 저장용
final_positions = []

for p in range(n_particles):
    X = [0]
    Y = [0]
    xt, yt = 0, 0

    for _ in range(n_steps):
        xstep = random.choice([-1, 1])
        ystep = random.choice([-1, 1])
        xt += xstep
        yt += ystep
        X.append(xt)
        Y.append(yt)

    # 경로를 선분으로 변환
    points = np.array([X, Y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    # 입자마다 LineCollection을 따로 생성하고 색 지정
    lc = LineCollection(segments, colors=[colors[p]], linewidth=2)
    ax.add_collection(lc)

    # 마지막 위치 저장
    final_positions.append((X[-1], Y[-1]))

# 시작점 표시
ax.plot(0, 0, marker='o', color='red', markersize=10, label='Start (0,0)')

# 각 입자의 마지막 위치 표시
for i, (x, y) in enumerate(final_positions):
    ax.plot(x, y, marker='X', color=colors[i], markersize=8)

ax.set_title("Multiple 2D Random Walks (Each Particle Colored)")
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_aspect('equal')

# 범위 자동 조절
all_x = [pos[0] for pos in final_positions] + [0]
all_y = [pos[1] for pos in final_positions] + [0]
ax.set_xlim(-nstep/15, nstep/15)
ax.set_ylim(-nstep/15, nstep/15)

plt.show()


# **1D Random walk을 통해 배우는 확산 모델**

이 코드는 **1차원 무작위 보행(1D Random Walk)**을 이용하여, 입자 확산 현상을 모사하고 이해하기 위한 간단한 시뮬레이션입니다.
모든 입자는 동일한 조건에서 독립적으로 움직이며, 각 입자의 최종 위치의 평균과 분산을 계산합니다.

* n_particles: 총 입자의 개수.
* n_steps: 각 입자의 이동 횟수. (시뮬레이션 시간에 해당합니다.)
* step_size: 한 번의 스텝에서 입자가 이동하는 거리.

In [ ]:
import random
import numpy as np

n_particles = 1000   # 입자 수
n_steps = 300      # 스텝 수
step_size = 1

# 모든 입자의 시간별 위치를 저장하는 2차원 리스트
X = [[0] for _ in range(n_particles)]  # 초기값 0 설정

# 랜덤 워크 진행
for p in range(n_particles):
    for _ in range(n_steps):
        step = random.choice([-step_size, step_size])
        new_pos = X[p][-1] + step  # 이전 위치 + 이동
        X[p].append(new_pos)

# 각 입자의 마지막 위치 추출
final_positions = [trajectory[-1] for trajectory in X]

# NumPy 배열로 변환
positions = np.array(final_positions)

# 통계 계산
mean_position = np.mean(positions)
variance_position = np.var(positions)

print(f"입자들의 최종 위치 평균: {mean_position:.2f}")
print(f"입자들의 최종 위치 분산: {variance_position:.2f}")


**1D Random Walk: 평균 위치와 분산의 시간 변화**

이 시뮬레이션은 통계적으로 다음과 같은 결과를 예측할 수 있습니다:

평균: ⟨𝑥⟩≈0

→ 이동 방향이 무작위이므로 평균 위치는 원점 근처에 머뭅니다.

분산: ⟨$𝑥^2$⟩−⟨𝑥⟩$^2$=2𝐷𝑡 (𝐷=(step size)$^2$/2)

→ 분산의 크기는 시간에 비례하며, 기울기는 **확산 계수**에 의해 결정됩니다.

**평균 변위 제곱** (3차원):

⟨$\Delta r^2(t)$⟩=2d𝐷𝑡

r: 입자의 위치 (r=(x,y,z))

D: 확산 계수

d: 공간의 차원 (1차원, 2차원, 3차원)


In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np

n_particles = 1000   # 입자 수
n_steps = 1000        # 스텝 수
step_size = 1

# 모든 입자의 시간별 위치를 저장하는 2차원 리스트
X = [[0] for _ in range(n_particles)]  # 초기값 0 설정

# 랜덤 워크 진행
for p in range(n_particles):
    for _ in range(n_steps):
        step = random.choice([-step_size, step_size])
        new_pos = X[p][-1] + step  # 이전 위치 + 이동
        X[p].append(new_pos)

# 시간에 따른 평균과 분산 계산
mean_over_time = []
var_over_time = []

# 시간 순서대로 위치 리스트 추출 (T개의 시간에 대해 반복)
for t in range(n_steps + 1):  # 0부터 n_steps까지 총 n_steps+1개
    positions_at_t = [X[p][t] for p in range(n_particles)]
    mean_over_time.append(np.mean(positions_at_t))
    var_over_time.append(np.var(positions_at_t))

# 그래프 그리기
time = np.arange(n_steps + 1)

plt.figure(figsize=(12, 5))

# 평균 위치 그래프
plt.subplot(1, 2, 1)
plt.ylim(-10, 10)
plt.plot(time, mean_over_time, color='blue')
plt.axhline(0, color='black', linestyle='dashed', linewidth=1.5, label='y = 0')  # y = 0 가이드라인
plt.xlabel('Time Step')
plt.ylabel('Mean Position')

diffusion_coefficient = step_size*step_size/2.

# 분산 그래프
plt.subplot(1, 2, 2)
plt.plot(time, var_over_time, color='green')
plt.plot(time, 2*diffusion_coefficient*time, color='black', linestyle='dashed', linewidth=1.5, label='y = x')  # y = x 가이드라인
plt.xlabel('Time Step')
plt.ylabel('Variance')

plt.tight_layout()
plt.show()


**1D Random Walk: 위치 분포의 확률밀도 함수 (PDF)**

이 시뮬레이션은 1차원에서의 랜덤 워크(random walk) 를 통해 많은 입자들의 최종 위치가 어떤 확률 분포를 따르는지를 살펴보는 데 목적이 있습니다.

각 입자의 최종 위치를 수집한 후, 절댓값 기준으로 데이터를 정리합니다.
→ 이는 랜덤 워크의 대칭성을 활용해 양의 방향에 대해서만 분포를 확인하기 위함입니다.

* np.histogram을 이용해 확률 밀도 함수(PDF)를 계산하고, matplotlib의 bar로 시각화합니다.

* 이론적으로, 충분한 스텝이 지나면 위치의 분포는 평균 0, 분산 2𝐷𝑡를 갖는 가우시안 분포에 수렴합니다.
→ 여기서 𝐷=(step_size)$^2$/2

1차원 확률밀도 함수:

### **PDF(x,t) = $\frac{1}{\sqrt{4 π Dt}}e^{-\frac{x^2}{4Dt}}$**




In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

n_particles = 1000
n_steps = 500
step_size = 1

# 입자별 시간별 위치 기록
X = [[0] for _ in range(n_particles)]

for p in range(n_particles):
    for _ in range(n_steps):
        step = random.choice([-step_size, step_size])
        new_pos = X[p][-1] + step
        X[p].append(new_pos)

# 마지막 위치 수집
final_positions = [trajectory[-1] for trajectory in X]

# 절댓값 기준 양수 쪽만 집계 (대칭 활용)
abs_positions = [abs(pos) for pos in final_positions]

# 히스토그램 계산 (density=True)
counts, bins = np.histogram(abs_positions, bins=20, density=True)
counts = counts / 2  # ← 확률 밀도를 절반으로 줄임

# bin 중심 계산
bin_centers = 0.5 * (bins[:-1] + bins[1:])

# 시각화: 히스토그램 수동으로 plot
plt.bar(bin_centers, counts, width=(bins[1] - bins[0]), alpha=0.6,
        color='skyblue', edgecolor='black', label='Simulation')


diffusion_coefficient = step_size*step_size/2.

# 정규분포 가이드라인 추가
x = np.linspace(min(bins), max(bins), 500)
pdf = norm.pdf(x, 0, (2*diffusion_coefficient*n_steps)**0.5)
plt.plot(x, pdf, 'r--', linewidth=2, label='Gaussian Fit')

# 그래프 꾸미기
plt.title(f't = {n_steps}')
plt.xlabel('Position')
plt.ylabel('Probability Density')
plt.yscale('log')
plt.grid(True)
plt.show()

# **과제: 1D Random Walk의 확산 특성 분석**


**목표**

1차원 랜덤 워크에서 스텝 크기(step size) 가 입자의 확산에 어떤 영향을 미치는지 분석합니다.

다음의 세 가지 step size 값을 사용하여 시뮬레이션을 반복합니다:

* step_size = 1

* step_size = 5

* step_size = 10

각 경우에 대해 다음을 수행하세요:

**1. 분산(Variance)의 시간 변화 그래프**

x축: 시간 (t)

y축: 각 시간에서의 분산

서로 다른 step_size 에 대한 결과를 **한 그래프**에 함께 표시하여 비교합니다.

그래프 위에 step_size 별 레전드도 표시하세요.

**2. 확률 밀도 함수 (PDF)의 비교**
특정 시간 (t = 500 등)에서의 입자들의 최종 위치 분포를 시각화하세요.

x축: 위치

y축: 확률 밀도 (Probability Density)

step_size 값에 따른 분포를 한 그래프에 겹쳐서 표시합니다.

각 분포에 대해 이론적인 정규분포 곡선도 함께 나타내 비교합니다.

------------------------------------------------------------

**그래프 구성 팁**

각 step_size 값에 대해 색상이나 스타일을 다르게 표현하세요.

**제출 형식**

Jupyter Notebook 파일 제출 또는 각 그래프에 대한 이미지 파일 제출

필요하다면 for 반복문을 이용해 여러 step_size 값을 한 번에 처리할 수 있도록 구현해보세요.